# Regulatory Compliance Analyzer using FalkorDB's GraphRAG SDK

In [56]:
# Import necessary libraries
import json
from graphrag_sdk.source import URL, JSONL
from graphrag_sdk import KnowledgeGraph, Ontology
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.litellm import LiteModel
from graphrag_sdk.model_config import KnowledgeGraphModelConfig

## Define data sources for regulatory compliance

In [89]:
gdpr_sources = [
    URL("https://ec.europa.eu/info/law/law-topic/data-protection_en") 
]

nist_sources = [
    URL("https://www.fortinet.com/resources/cyberglossary/nist-compliance")
]

soc2_sources = [
    URL("https://www.vanta.com/collection/soc-2/soc-2-compliance-requirements")
]

## Initialize the model

In [77]:
model = LiteModel(model_name="gemini/gemini-2.0-flash-exp")

## Create ontologies from sources

In [59]:
# Function to create an ontology from sources
def create_ontology(sources, ontology_name):
    ontology = Ontology.from_sources(
        sources=sources,
        model=model,
    )
    with open(f"{ontology_name}.json", "w") as f:
        json.dump(ontology.to_json(), f)
    return ontology

In [ ]:
# Create ontologies for GDPR, NIST, and SOC 2
gdpr_ontology = create_ontology(gdpr_sources, "gdpr_ontology")
print("GDPR Ontology:", gdpr_ontology)

GDPR Ontology: Entities:
- (:EuropeanCommission {type: "string*", name: "string!*"})
- (:DataProtection {description: "string!*"})
- (:EULaw {description: "string!*"})
- (:GDPR {full_name: "string!*", short_name: "string"})
- (:LED {full_name: "string!*", short_name: "string"})
- (:EUDPR {full_name: "string!*", short_name: "string"})
- (:ThirdCountry {name: "string!*"})
- (:SCC {full_name: "string!*", short_name: "string"})
- (:BCR {full_name: "string!*", short_name: "string"})
- (:NationalDataProtectionAuthority {country: "string!*"})
- (:TimelineEvent {date: "string*", description: "string!*"})
- (:Report {topic: "string*", date: "string*"})
- (:Communication {title: "string!*", date: "string*"})
- (:EuropeanParliament {type: "string*", name: "string!*"})
- (:Council {type: "string*", name: "string!*"})
- (:MemberState {name: "string!*"})
- (:SmallAndMediumSizedEnterprise {abbreviation: "string!*"})
- (:EuLaw {name: "string!*"})
- (:Gdpr {full_name: "string!*", short_name: "string"}

In [80]:
nist_ontology = create_ontology(nist_sources, "nist_ontology")
print("NIST Ontology:", nist_ontology)

NIST Ontology: Entities:
- (:Nist {name: "string!*", description: "string"})
- (:Guideline {name: "string!*", description: "string"})
- (:Standard {name: "string!*", description: "string"})
- (:Organization {name: "string!*", description: "string"})
- (:Act {name: "string!*", description: "string"})
- (:Tool {name: "string!*", description: "string"})
- (:System {name: "string!*", description: "string"})
- (:Control {name: "string!*", description: "string"})
- (:Framework {name: "string!*", description: "string"})
- (:Team {name: "string!*"})
- (:Data {name: "string!*"})
- (:U_s_government {name: "string!*"})
- (:Fortinet {name: "string!*"})
- (:Vulnerability {name: "string!*"})
- (:Threat {name: "string!*"})
- (:It_ot {name: "string!*"})

Edges:
- (:Organization)-[:COMPLIES_WITH {}]->(:Nist)
- (:Nist)-[:GOVERNS {}]->(:Guideline)
- (:Team)-[:USES {}]->(:Tool)
- (:Nist)-[:PROTECTS {}]->(:Data)
- (:Nist)-[:ENSURES {}]->(:Control)
- (:Nist)-[:PART_OF {}]->(:U_s_government)
- (:Fortinet)-

In [81]:
soc2_ontology = create_ontology(soc2_sources, "soc2_ontology")
print("SOC2 Ontology:", soc2_ontology)

SOC2 Ontology: Entities:
- (:Product {name: "string!*", description: "string"})
- (:Framework {name: "string!*", description: "string"})
- (:Company {name: "string!*", description: "string"})
- (:Service {name: "string!*", description: "string"})
- (:Compliance {type: "string!*", name: "string!*", description: "string"})
- (:Report {name: "string!*", type: "string*", report_type: "string*", description: "string"})
- (:Criterion {name: "string!*", description: "string"})
- (:Security {description: "string", level: "string!*"})
- (:Automation {description: "string", name: "string!*"})
- (:RiskManagement {description: "string", name: "string!*"})
- (:Organization {description: "string", name: "string!*"})
- (:Trust {description: "string", name: "string!*"})
- (:Data {type: "string", data_type: "string!*", description: "string"})
- (:Auditor {type: "string", name: "string!*", description: "string"})
- (:Control {description: "string", name: "string!*"})
- (:Integration {description: "stri

## Create knowledge graphs

In [82]:
# Function to create a knowledge graph
def create_knowledge_graph(ontology, name):
    return KnowledgeGraph(
        name=name,
        model_config=KnowledgeGraphModelConfig.with_model(model),
        ontology=ontology,
        host="127.0.0.1",
        port=6379
    )


In [ ]:
# Create knowledge graphs for GDPR, NIST, and SOC 2
gdpr_kg = create_knowledge_graph(gdpr_ontology, "gdpr_compliance")
nist_kg = create_knowledge_graph(nist_ontology, "hipaa_compliance")
soc2_kg = create_knowledge_graph(soc2_ontology, "soc2_compliance")

In [ ]:
# Process sources for each knowledge graph
gdpr_kg.process_sources(gdpr_sources)

In [ ]:
nist_kg.process_sources(nist_sources)

In [ ]:
soc2_kg.process_sources(soc2_sources)

## Create agents

In [86]:
# Create agents for GDPR, NIST, and SOC 2 compliance analysis

gdpr_agent = KGAgent(
    agent_id="gdpr_agent",
    kg=gdpr_kg,
    introduction="I specialize in GDPR compliance analysis for European data protection regulations."
)

nist_agent = KGAgent(
    agent_id="nist_agent",
    kg=nist_kg,
    introduction="I specialize in NIST compliance analysis, ensuring adherence to cybersecurity and risk management frameworks."
)

soc2_agent = KGAgent(
    agent_id="soc2_agent",
    kg=soc2_kg,
    introduction="I specialize in SOC 2 compliance analysis, focusing on security, availability, processing integrity, confidentiality, and privacy for service organizations."
)

# Create an orchestrator for compliance analysis
compliance_orchestrator = Orchestrator(
    model=model,
    backstory="You are a compliance analyst who determines if organizations meet GDPR, NIST, and/or SOC 2 requirements."
)

# Register the agents with the orchestrator
compliance_orchestrator.register_agent(gdpr_agent)
compliance_orchestrator.register_agent(nist_agent)
compliance_orchestrator.register_agent(soc2_agent)

## Check compliance based on company information

In [87]:
# Function to check compliance based on company information
def check_compliance(company_info):
    
    query = f"""
    COMPANY PROFILE:
    {company_info}
    
     COMPLIANCE ANALYSIS TASKS:
    1. Identify the main regulatory standards applicable to the company (e.g., GDPR, NIST, SOC 2).
    2. Briefly assess whether the company appears to be in compliance with these regulations based on the available information.
    3. Mention any obvious gaps or areas where the company may potentially not meet regulatory requirements.
    4. Provide general recommendations on how the company could improve compliance with necessary regulations.
    5. Evaluate the company's data handling practices, cross-border data transfers, and security measures, and indicate if these practices align with regulatory expectations.
    6. Suggest potential areas of improvement in privacy policies, consent mechanisms, and risk mitigation strategies.
    """
    
    runner = compliance_orchestrator.ask(query)
    return runner.output


In [ ]:
# Load company information
company_info = JSONL("company_info.jsonl")

In [88]:
# Run compliance check
print(check_compliance(company_info))

Based on the provided company profile and compliance analysis tasks, here's a breakdown of the applicable regulatory standards and compliance considerations:

1.  **GDPR (General Data Protection Regulation):** According to the GDPR agent, the General Data Protection Directive is for Small and Medium Sized Enterprises. Therefore, GDPR requirements should be considered if the company falls into this category.
2.  **NIST (National Institute of Standards and Technology):** The NIST agent indicated that Fortinet and U.S.-based organizations comply with The National Institute of Standards and Technology (NIST). NIST is a body that handles the technology, metrics, and standards used within the technology and science industries. Therefore, NIST compliance should be considered if the company is based in the U.S. or works in the technology or science industries.
3.  **SOC 2 (Service Organization Control 2):** The SOC 2 agent identified that the company profile includes information related to bus